In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

def science_plot(fontsize = 9):
    import scienceplots
    plt.style.use(['science','grid','notebook'])
    plt.rcParams.update({
        'font.size'       : fontsize,    # General font size
        'axes.titlesize'  : fontsize,    # Font size of the axes title
        'axes.labelsize'  : fontsize,    # Font size of the axes labels
        'xtick.labelsize' : fontsize,    # Font size of the x-axis tick labels
        'ytick.labelsize' : fontsize,    # Font size of the y-axis tick labels
        'legend.fontsize' : fontsize,    # Font size of the legend
        'figure.titlesize': fontsize,    # Font size of the figure title
        'legend.fancybox' : False,       # Disable the fancy box for legend
        'legend.edgecolor': 'k',         # Set legend border color to black
        'text.usetex'     : True,        # Use LaTeX for text rendering
        'font.family'     : 'serif'      # Set font family to serif
    })
science_plot()

In [2]:
# Data: lengths in cm, periods in s
data = {
    "Length (cm)": [51.2, 59.7, 68.2, 79.7, 88.3],
    "Period (s)": [1.448, 1.566, 1.669, 1.804, 1.896]
}
# Create DataFrame
df = pd.DataFrame(data)
# Compute g = 4π²l / T²
df["g (cm/s²)"] = 4 * np.pi**2 * df["Length (cm)"] / df["Period (s)"]**2
# Mean, Standard Deviation (SD), and Standard Deviation of the Mean (SDOM)
mean_g = df["g (cm/s²)"].mean()
std_g = df["g (cm/s²)"].std(ddof=1)
sdom_g = std_g / np.sqrt(len(df))
print(f"g = {mean_g:.2f} ± {sdom_g:.2f} cm/s²")

g = 965.64 ± 1.46 cm/s²


In [5]:
accepted_g = 979.6
discrepancy = abs(mean_g - accepted_g)
print(f"Discrepancy = {discrepancy:.2f} cm/s², {discrepancy/sdom_g:.2f} times SDOM")
print(f"Is discrepancy near 10 SDOM? {'Yes' if discrepancy > 10 * sdom_g * (0.95) else 'No'}")


Discrepancy = 13.96 cm/s², 9.59 times SDOM
Is discrepancy near 10 SDOM? Yes


In [9]:
# Compute the required effective lengths
df["l_eff (cm)"] = accepted_g * df["Period (s)"]**2 / (4 * np.pi**2)
# Compute percent correction
df["% Error in l"] = 100 * (df["l_eff (cm)"] - df["Length (cm)"]) / df["Length (cm)"]
avg_percent_error = df["% Error in l"].mean()
print(f"Required average correction in l ≈ {avg_percent_error:.2f}%")

Required average correction in l ≈ 1.45%


In [11]:
# Correct length: add 1.00 cm (half of 2.00 cm diameter)
df["Corrected l (cm)"] = df["Length (cm)"] + 1.00
# Recalculate g with corrected length
df["Corrected g (cm/s²)"] = 4 * np.pi**2 * df["Corrected l (cm)"] / df["Period (s)"]**2
# Recalculate mean and uncertainty
corrected_mean_g = df["Corrected g (cm/s²)"].mean()
corrected_std_g = df["Corrected g (cm/s²)"].std(ddof=1)
corrected_sdom_g = corrected_std_g / np.sqrt(len(df))
print(f"Corrected g = {corrected_mean_g:.2f} ± {corrected_sdom_g:.2f} cm/s²")

Corrected g = 980.08 ± 0.96 cm/s²


In [14]:
import numpy as np
from scipy.stats import norm
import pandas as pd

# Sample sizes
N_values = [5, 10, 15, 20, 50, 100, 200, 1000]

# Compute z using Chauvenet's criterion
z_values = [norm.ppf(1 - 1/(2*N)) for N in N_values]

# Create a table
table = pd.DataFrame({
    'N': N_values,
    'Chauvenet z-threshold': np.round(z_values, 4)
})

print(table)


      N  Chauvenet z-threshold
0     5                 1.2816
1    10                 1.6449
2    15                 1.8339
3    20                 1.9600
4    50                 2.3263
5   100                 2.5758
6   200                 2.8070
7  1000                 3.2905


In [ ]:
import numpy as np
from scipy.optimize import curve_fit
# Data
Rx = np.array([0, 20, 40, 60, 80, 100])
I = np.array([0.374, 0.333, 0.303, 0.281, 0.263, 0.249])
V = 25  # Volts
# Model function: I(Rx) = V * [1/R1 + 1/(R2 + Rx)]
def current_model(Rx, R1, R2):
    return V * (1/R1 + 1/(R2 + Rx))
# Nonlinear least squares fitting
popt, pcov = curve_fit(current_model, Rx, I)
R1_fit, R2_fit = popt
R1_err, R2_err = np.sqrt(np.diag(pcov))
# Output best-fit values and uncertainties
print(f"Best-fit R1 = {R1_fit:.2f} ± {R1_err:.2f} ohms")
print(f"Best-fit R2 = {R2_fit:.2f} ± {R2_err:.2f} ohms")


Best-fit R1 = 201.04 ± 0.57 ohms
Best-fit R2 = 100.02 ± 0.24 ohms


In [16]:
import numpy as np
from scipy.optimize import curve_fit

# Original data
Rx_data = np.array([0, 20, 40, 60, 80, 100])
I_data = np.array([0.374, 0.333, 0.303, 0.281, 0.263, 0.249])
V = 25  # Volts

# Model: I(Rx) = V * (1/R1 + 1/(R2 + Rx))
def current_model(Rx, R1, R2):
    return V * (1/R1 + 1/(R2 + Rx))

# Initial nonlinear fit
popt, _ = curve_fit(current_model, Rx_data, I_data)
R1_init, R2_init = popt

# Bootstrap
n_bootstrap = 10000
R1_samples = []
R2_samples = []
rng = np.random.default_rng()

for _ in range(n_bootstrap):
    indices = rng.integers(0, len(Rx_data), len(Rx_data))
    Rx_boot = Rx_data[indices]
    I_boot = I_data[indices]
    try:
        popt_boot, _ = curve_fit(current_model, Rx_boot, I_boot, p0=[R1_init, R2_init])
        R1_samples.append(popt_boot[0])
        R2_samples.append(popt_boot[1])
    except RuntimeError:
        continue  # skip fits that don't converge

R1_samples = np.array(R1_samples)
R2_samples = np.array(R2_samples)

# Compute 95% confidence intervals
R1_ci = np.percentile(R1_samples, [2.5, 97.5])
R2_ci = np.percentile(R2_samples, [2.5, 97.5])

# Results
print(f"R1: 95% CI = [{R1_ci[0]:.2f}, {R1_ci[1]:.2f}] ohms")
print(f"R2: 95% CI = [{R2_ci[0]:.2f}, {R2_ci[1]:.2f}] ohms")


C:\Users\verci\AppData\Local\Temp\ipykernel_14384\2892399375.py:28: OptimizeWarning: Covariance of the parameters could not be estimated
  popt_boot, _ = curve_fit(current_model, Rx_boot, I_boot, p0=[R1_init, R2_init])


R1: 95% CI = [199.68, 202.43] ohms
R2: 95% CI = [99.06, 100.46] ohms
